In [ ]:
import os
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker

import rasterio
import shapely
from shapely.geometry import LineString, Polygon

import flopy
from flopy.discretization import StructuredGrid, VertexGrid
from flopy.utils.triangle import Triangle
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.gridgen import Gridgen
import flopy.plot.styles as styles

In [ ]:
# import all plot style information from defaults.py
from defaults import *

### Conversion factors

In [ ]:
ft2m = 1. / 3.28081
ft3tom3 = 1. * ft2m * ft2m * ft2m
ftpd2cmpy = 1000. * 365.25 * ft2m
mpd2cmpy = 100. * 365.25
mpd2inpy = 12. * 365.25 * 3.28081

### Load raster data

In [ ]:
kaq = flopy.utils.Raster.load("./data/k_aq_SI.tif")
kclay = flopy.utils.Raster.load("./data/k_clay_SI.tif")
top_base = flopy.utils.Raster.load("./data/top_SI.tif")
bot = flopy.utils.Raster.load("./data/bottom_SI.tif")
lake_location = flopy.utils.Raster.load("./data/lake_location_SI.tif")

### create a structured grid for plotting

In [ ]:
nlay, nrow, ncol = 6, 40, 25
shape2d = (nrow, ncol)
shape3d = (nlay, nrow, ncol)
delr = delc = 500.0 * ft2m
delrs = np.ones(ncol, dtype=np.float32) * delr
delcs = np.ones(nrow, dtype=np.float32) * delc
top = top_base.get_array(1)
botm = np.zeros(shape3d, dtype=np.float32)
botm[0] = -5.0 * ft2m
botm[1] = -50.0 * ft2m
botm[2] = -51.0 * ft2m
botm[3] = -100.0 * ft2m
botm[4] = bot.get_array(1)

In [ ]:
struct_grid = flopy.discretization.StructuredGrid(
    nlay=1,
    nrow=nrow,
    ncol=ncol,
    delr=delrs,
    delc=delcs,
    top=top,
    botm=botm[0].reshape((1, nrow, ncol)),
)

### define the features (in feet)

In [ ]:
mv_boundary = """0.0 0.0
0.0 20000.0
12500.0 20000.0
12500.0 0.0"""

In [ ]:
river = """4250.0 8750.0 
4250.0 0.0"""

In [ ]:
river_box = """3500.0 0.0
3500.0 9500.0
5000.0 9500.0
5000.0 0.0"""

In [ ]:
wells = """7250. 17250.
7750. 2750.
2750 3750."""

In [ ]:
lake = """1500. 18500.
3500. 18500.
3500. 15500.
4000. 15500.
4000. 14500.
4500. 14500.
4500. 12000.
2500. 12000.
2500. 12500.
2000. 12500.
2000. 14000.
1500. 14000.
1500. 15000.
1000. 15000.
1000. 18000.
1500. 18000."""

In [ ]:
# in meters
boundary_refinement = 100.0
river_refinement = 25.0
lake_refinement = 30.0 
max_boundary_area = boundary_refinement * boundary_refinement
max_river_area = river_refinement * river_refinement
max_lake_area = lake_refinement * lake_refinement

In [ ]:
boundary_polygon = string2geom(mv_boundary, conversion=ft2m)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)
bp_densify = np.array(densify_geometry(bp, boundary_refinement))

In [ ]:
river_polyline = string2geom(river, conversion=ft2m)
sg = np.array(river_polyline)
sg_densify = np.array(densify_geometry(sg, river_refinement))

In [ ]:
river_boundary = string2geom(river_box, conversion=ft2m)
rb = np.array(river_boundary)
rb_densify = np.array(densify_geometry(rb, river_refinement))

In [ ]:
lake_polygon = string2geom(lake, conversion=ft2m)
lake_plot = string2geom(lake, conversion=ft2m)
lake_plot += [lake_plot[0]]
lake_plot = np.array(lake_plot)
lp = np.array(lake_polygon)
lp_densify = np.array(densify_geometry(lp, lake_refinement))

In [ ]:
well_points = string2geom(wells, conversion=ft2m)
wp = np.array(well_points)

#### simple plotting functions

In [ ]:
def plot_wells(ax=None, ms=None):
    if ax is None:
        ax = plt.gca()
    ax.plot(wp[:, 0], wp[:, 1], "ro", ms=ms)
    return ax

In [ ]:
def plot_river(
    ax=None,
):
    if ax is None:
        ax = plt.gca()
    ax.plot(sg_densify[:, 0], sg_densify[:, 1], **river_dict)
    return ax

In [ ]:
def plot_lake(ax=None, lw=0.5, color="cyan", marker=None, densify=False):
    if ax is None:
        ax = plt.gca()
    if densify:
        arr = lp_densify
    else:
        arr = lake_plot
    ax.plot(arr[:, 0], arr[:, 1], ls="-", color=color, lw=lw, marker=marker)
    return ax

In [ ]:
def plot_circles(ax=None, radius=1.0, lw=0.5, color="red"):
    if ax is None:
        ax = plt.gca()
    for pt in wp:
        center = (pt[0], pt[1])
        pts = circle_function(center=center, radius=radius)
        ax.plot(pts[:, 0], pts[:, 1], ls=":", color=color, lw=lw)
    return ax

### plot the features

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
ax.set_aspect("equal")

ax.plot(bp_densify[:, 0], bp_densify[:, 1], "ko-")
plot_lake(ax, marker="o", densify=True)
plot_river(ax)
plot_wells(ax)
plot_circles(ax, radius=300);

### use Triangle to create mesh

In [ ]:
temp_path = "triangle_data"

In [ ]:
maximum_area = 150. * 150.
well_dv = 300.0

In [ ]:
tri = Triangle(
    angle=30,
    nodes=sg_densify,
    model_ws=temp_path,
)
tri.add_polygon(bp_densify)
tri.add_polygon(rb_densify)
tri.add_polygon(lp_densify)
tri.add_region((10, 10), attribute=10, maximum_area=max_boundary_area)
tri.add_region(
   (3050.0, 3050.0), attribute=10, maximum_area=max_boundary_area,)
tri.add_region((900.0, 4600.0), attribute=11, maximum_area=max_lake_area)
tri.add_region((1200.0, 150.0), attribute=10, maximum_area=max_river_area)
for idx, w in enumerate(wp):
    center = (w[0], w[1])
    tri.add_polygon(circle_function(center=center, radius=100.0))
    tri.add_region(center, attribute=idx, maximum_area=500.0)
tri.build(verbose=False)

In [ ]:
tri.plot(lw=0.5)
ax = plot_wells(ms=3)
plot_river(ax)

### convert triangular mesh to a voronoi grid

In [ ]:
vor = VoronoiGrid(tri)

In [ ]:
vor.plot()
ax = plot_wells(ms=3)

### create a flopy Vertex grid to use for the model

In [ ]:
gridprops = vor.get_gridprops_vertexgrid()
idomain = np.ones((1, vor.ncpl), dtype=int)
voronoi_grid = VertexGrid(**gridprops, nlay=1, idomain=idomain)

In [ ]:
voronoi_grid.plot()

### intersect the rasters with the vertex grid

In [ ]:
top_vg = top_base.resample_to_grid(
    voronoi_grid,
    band=top_base.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
bot_vg = bot.resample_to_grid(
    voronoi_grid,
    band=bot.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
lake_cells_vg = lake_location.resample_to_grid(
    voronoi_grid,
    band=lake_location.bands[0],
    method="nearest",
    extrapolate_edges=True,
)

In [ ]:
kaq_vg = kaq.resample_to_grid(
    voronoi_grid,
    band=kaq.bands[0],
    method="nearest",
    extrapolate_edges=True,
)

In [ ]:
kclay_vg = kclay.resample_to_grid(
    voronoi_grid,
    band=kclay.bands[0],
    method="nearest",
)

In [ ]:
np.unique(kaq_vg)

In [ ]:
np.unique(kclay_vg)

In [ ]:
# create confining unit location map
kclay_loc_vg = np.zeros(kclay_vg.shape, dtype=int)
kclay_loc_vg[kclay_vg < 60.] = 1

In [ ]:
kclay_vg.min(), kclay_vg.max(), kclay.get_array(1).min(), kclay.get_array(1).max()

In [ ]:
idomain_2 = np.ones(kclay_vg.shape, dtype=int)
idomain_2[kclay_loc_vg == 0] = -1

In [ ]:
porosity_2 = np.full(kclay_vg.shape, 0.2, dtype=float)
porosity_2[kclay_loc_vg == 1] = 0.4

In [ ]:
vmin, vmax = kclay.get_array(1).min(), kclay.get_array(1).max()
with styles.USGSMap():
    fig, ax = plt.subplots(
        1,
        1,
        sharey=True,
        # figsize=one_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    cb = mm.plot_array(idomain_2)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    plt.colorbar(cb, ax=ax, shrink=0.5)

### set the bottom of each layer

In [ ]:
# zero-based variable names
bot_l2 = np.full(bot_vg.shape, -51.0 * ft2m, dtype=float)
bot_l3 = np.full(bot_vg.shape, -100.0 * ft2m, dtype=float)
bot_l4 = bot_vg + 0.5 * (bot_l3 - bot_vg)

In [ ]:
bot_l2[idomain_2] = -50. * ft2m

In [ ]:
botm = [-5. * ft2m, -50. * ft2m, bot_l2, -100.0 * ft2m, bot_l4, bot_vg]
botm[0:4]

### create a model grid for the lake

In [ ]:
lake_grid_top = np.full((vor.ncpl), 50.0, dtype=float)
lake_vg_grid = flopy.discretization.VertexGrid(**gridprops, nlay=1, idomain=idomain, top=lake_grid_top, botm=top_vg.reshape(1, vor.ncpl))

### plot the resampled rasters

In [ ]:
two_panel_figsize = (17.15 / 2.541, 0.8333 * 17.15 / 2.541)
one_panel_figsize = (8.25 / 2.541, 13.2 / 2.541)
six_panel_figsize = (17.15 / 2.541, 1.4 * 0.8333 * 17.15 / 2.541)

In [ ]:
vmin, vmax = 1, 20
with styles.USGSMap():
    fig, [ax1, ax2] = plt.subplots(
        1,
        2,
        sharey=True,
        figsize=two_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    ax = ax1
    mm = flopy.plot.PlotMapView(modelgrid=struct_grid, ax=ax)
    mm.plot_array(top_base.get_array(1), vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        top_base.get_array(1),
        **mv_contour_dict,
    )
    ax.clabel(cs, **clabel_dict)

    ax = ax2
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    mm.plot_array(top_vg.reshape(1, voronoi_grid.ncpl), vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        top_vg,
        **mv_contour_dict,
    )
    ax.clabel(cs, **clabel_dict)

    fpth = "figures/mv_voronoi_top.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
vmin, vmax = -75, -50
with styles.USGSMap():
    fig, [ax1, ax2] = plt.subplots(
        1,
        2,
        sharey=True,
        figsize=two_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    ax = ax1
    mm = flopy.plot.PlotMapView(modelgrid=struct_grid, ax=ax)
    mm.plot_array(bot.get_array(1), vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    ax = ax2
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    mm.plot_array(bot_vg, vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    fpth = "figures/mv_voronoi_bot.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
with styles.USGSMap():
    fig, [ax1, ax2] = plt.subplots(
        1,
        2,
        sharey=True,
        figsize=two_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    ax = ax1
    mm = flopy.plot.PlotMapView(modelgrid=struct_grid, ax=ax)
    mm.plot_array(lake_location.get_array(1))
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    ax = ax2
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    mm.plot_array(lake_cells_vg)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    fpth = "figures/mv_voronoi_lake.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
vmin, vmax = kaq.get_array(1).min(), kaq.get_array(1).max()
with styles.USGSMap():
    fig, [ax1, ax2] = plt.subplots(
        1,
        2,
        sharey=True,
        figsize=two_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    ax = ax1
    mm = flopy.plot.PlotMapView(modelgrid=struct_grid, ax=ax)
    mm.plot_array(kaq.get_array(1), vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    ax = ax2
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    pa = mm.plot_array(kaq_vg, vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    
    axs = np.array([ax1, ax2])
    cb = plt.colorbar(
    pa,
    ax=axs[:], # select all axis from second column
    orientation='horizontal',
    shrink=0.75)

    fpth = "figures/mv_voronoi_kaq.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
vmin, vmax = kclay.get_array(1).min(), kclay.get_array(1).max()
with styles.USGSMap():
    fig, [ax1, ax2] = plt.subplots(
        1,
        2,
        sharey=True,
        figsize=two_panel_figsize,
        constrained_layout=True,
        subplot_kw=dict(aspect="equal"),
    )
    ax = ax1
    mm = flopy.plot.PlotMapView(modelgrid=struct_grid, ax=ax)
    mm.plot_array(kclay.get_array(1), vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    ax = ax2
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    pa = mm.plot_array(kclay_vg, vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)

    axs = np.array([ax1, ax2])
    cb = plt.colorbar(
    pa,
    ax=axs[:], # select all axis from second column
    orientation='horizontal',
    shrink=0.75)    
    
    fpth = "figures/mv_voronoi_kclay.png"
    plt.savefig(fpth, dpi=300);

### intersect features with the grid

In [ ]:
ixs = flopy.utils.GridIntersect(voronoi_grid, method="vertex")
sg_result = ixs.intersect(LineString(sg_densify), sort_by_cellid=False)

In [ ]:
sg_result.shape, sg_result["lengths"].sum(), sg_result.dtype.names

In [ ]:
sg_result["vertices"][0:5]

In [ ]:
flopy.plot.plot_shapefile(sg_result["ixshapes"])
ax = plt.gca()
ax.set_xlim(voronoi_grid.extent[0:2])
ax.set_ylim(voronoi_grid.extent[2:])

### forcing data

In [ ]:
rainfall = 0.00821918 * ft2m
evaporation = 0.0062296 * ft2m
net_rainfall = 0.003641 * ft2m

In [ ]:
rainfall, evaporation

In [ ]:
rainfall * mpd2cmpy, evaporation * mpd2cmpy, net_rainfall * mpd2cmpy

In [ ]:
rainfall * mpd2inpy, evaporation * mpd2inpy, net_rainfall * mpd2inpy

## Build SFR, LAK, DRN, and WEL datasets

SFR datasets

In [ ]:
sfr_unit_conversion = 1.0 * 86400.0  #1.486 * 86400.0

In [ ]:
sfr_plt_array = np.zeros(voronoi_grid.ncpl, dtype=int)

In [ ]:
sfr_nodes = np.arange(0, sg_result.shape[0])
gwf_nodes = sg_result["cellids"][::-1]
sfr_lengths = sg_result["lengths"][::-1]

In [ ]:
total_cond = 1800000.0 * ft3tom3
sfr_width = 10.0 * ft2m
sfr_bedthick = 1.0 * ft2m
sfr_hk = total_cond * sfr_bedthick / (sfr_width * sfr_lengths.sum())
total_cond, sfr_hk

In [ ]:
b0, b1 = -0.3 * ft2m, -2.05 * ft2m
sfr_slope = -0.0002
cum_dist = np.zeros(sfr_nodes.shape, dtype=float)
cum_dist[0] = 0.5 * sfr_lengths[0]
for idx in range(1, sfr_nodes.shape[0]):
    cum_dist[idx] = cum_dist[idx - 1] + 0.5 * (
        sfr_lengths[idx - 1] + sfr_lengths[idx]
    )
sfr_bot = b0 + sfr_slope * cum_dist
cum_dist[-1], sfr_lengths.sum(), sfr_lengths[-1], sfr_bot[-1]

In [ ]:
sfr_conn = []
for idx, node in enumerate(sfr_nodes):
    iconn = [node]
    if idx > 0:
        iconn.append(sfr_nodes[idx - 1])
    if idx < sfr_nodes.shape[0] - 1:
        iconn.append(-sfr_nodes[idx + 1])
    sfr_conn.append(iconn)

In [ ]:
# <rno> <cellid(ncelldim)> <rlen> <rwid> <rgrd> <rtp> <rbth> <rhk> <man> <ncon> <ustrf> <ndv>
sfrpak_data = []
for idx, (cellid, rlen, rtp) in enumerate(
    zip(gwf_nodes, sfr_lengths, sfr_bot)
):
    sfr_plt_array[cellid] = 1
    sfrpak_data.append(
        (
            idx,
            (
                0,
                cellid,
            ),
            rlen,
            sfr_width,
            -sfr_slope,
            rtp,
            sfr_bedthick,
            sfr_hk,
            0.030,
            (len(sfr_conn[idx]) - 1),
            1.0,
            0,
        )
    )
sfrpak_data[-2:]

In [ ]:
sfr_spd = [(node, "rainfall", rainfall) for node in sfr_nodes] + [
    (node, "evaporation", evaporation) for node in sfr_nodes
]

LAK datasets

In [ ]:
lak_length_conversion = 1.0  #3.28081
lak_time_conversion = 86400.0

In [ ]:
lake_ic = 11.3 * ft2m
idx = np.where(lake_cells_vg == 1.0)

lake_map = np.ones(voronoi_grid.ncpl, dtype=int) * -1
lake_map[idx] = 0

(idomain, lakpak_dict, lak_connections) = flopy.mf6.utils.get_lak_connections(
    voronoi_grid,
    lake_map,
    bedleak=0.0013,
)

# add concentration to lake data as aux
lakpak_data = [(0, lake_ic, lakpak_dict[0], 1.0)]
lake_spd = [
    (0, "rainfall", rainfall),
    (0, "evaporation", evaporation),
]

In [ ]:
lake_ic, lakpak_data

In [ ]:
lak_connections[0:10]

DRN datasets

In [ ]:
areas = []
for idx in range(voronoi_grid.ncpl):
    vertices = np.array(voronoi_grid.get_cell_vertices(idx))
    area = Polygon(vertices).area
    areas.append(area)

In [ ]:
0.1 * ft2m

In [ ]:
drn_kv = 0.1 * ft2m
drn_depth = 1.0 * ft2m
drn_spd = []
for idx, elev in enumerate(top_vg):
    if lake_cells_vg[idx] > 0:
        cond = drn_kv * areas[idx] / drn_depth
        drn_spd.append([(0, idx), elev, cond, -drn_depth])
drn_spd[:10]

WEL datasets

In [ ]:
well_points

In [ ]:
well_loc = []
for x, y in well_points:
    well_loc.append(voronoi_grid.intersect(x, y))
well_loc

In [ ]:
# first well is Virginia City well site 2
# second well is Reilly well
# third well is Virginia City well site 1
# boundname = ["VC2", "Reilly", "VC1"]
well_boundnames = ["P3", "P1", "P2"]
rates = [-1900.0, -7600.0, -7600.0]
welspd = [
    [nlay - 1, cellid, rates[idx], well_boundnames[idx]]
    for idx, cellid in enumerate(well_loc)
]
welspd

In [ ]:
p2_loc = tuple(welspd[-1][0:2])
p2_loc

In [ ]:
p2_loc = tuple(welspd[-1][0:2])

## Plot a series of grids

In [ ]:
def add_subdomain(ax, xll, xur, yll, yur, lw=0.75, color="black", text="B"):
    x = [xll, xll, xur, xur, xll]
    y = [yll, yur, yur, yll, yll]
    ax.plot(x, y, lw=lw, color=color)
    styles.add_text(
        ax=ax,
        text=text,
        x=xll,
        y=yur,
        color=color,
        transform=False,
        ha="right",
        va="bottom",
        italic=False,
    )

In [ ]:
def add_nodes(ax, xll, xur, yll, yur, edge=20.0, text_offset=2.0):
    for idx, (x, y) in enumerate(zip(xcv, ycv)):
        if x > xll and x < xur and y > yll and y < yur:
            ax.plot(x, y, marker=".", markersize=4, color="black")
            if (
                x > xll + edge
                and x < xur - edge
                and y > yll
                and y < yur - edge
            ):
                xt = x + 4 * text_offset
                yt = y + text_offset
                styles.add_text(
                    ax=ax,
                    x=xt,
                    y=yt,
                    text=f"{idx + 1}",
                    bold=False,
                    transform=False,
                    ha="center",
                    va="bottom",
                    fontsize=5,
                )

In [ ]:
def set_ticklabels(
    ax,
    fmt="{:.1f}",
    skip_xticklabels=False,
    skip_yticklabels=False,
    skip_xlabel=False,
    skip_ylabel=False,
    xticks=None,
    yticks=None,
):
    if xticks is None:
        labels = [ax.get_xticks().tolist()]
    else:
        ax.set_xticks(xticks, labels=[str(value) for value in xticks])
        labels = [xticks]

    if yticks is None:
        labels += [ax.get_yticks().tolist()]
    else:
        ax.set_yticks(yticks, labels=[str(value) for value in yticks])
        labels += [yticks]

    for idx, label in enumerate(labels):
        for jdx, value in enumerate(label):
            labels[idx][jdx] = fmt.format(float(value) / 1000.0)

    if skip_xticklabels:
        ax.set_xticklabels([])
    else:
        ax.xaxis.set_major_locator(mticker.FixedLocator(ax.get_xticks()))
        ax.set_xticklabels(labels[0])

    if skip_yticklabels:
        ax.set_yticklabels([])
    else:
        ax.yaxis.set_major_locator(mticker.FixedLocator(ax.get_yticks()))
        ax.set_yticklabels(labels[1])

    if not skip_xlabel:
        ax.set_xlabel("x position (km)")
    if not skip_ylabel:
        ax.set_ylabel("y position (km)")

In [ ]:
def plot_feature_labels(ax):
    styles.add_text(ax=ax, text="Blue\nLake", x=610, y=5000., transform=False, bold=False, ha="center", va="center")
    styles.add_text(ax=ax, text="Straight River", x=1425, y=1500., transform=False, bold=False, va="center", ha="center", rotation=90)
    for xy, name in zip(well_points, well_boundnames):
        styles.add_annotation(ax=ax, text=name, xy=xy, xytext=(-15, 10),
                              bold=False,
                              textcoords="offset points",
                              arrowprops=arrowprops,)

    return

A few coordinates for plotting

In [ ]:
xcv, ycv = voronoi_grid.xcellcenters, voronoi_grid.ycellcenters

In [ ]:
x0 = x1 = sg[:, 0].min()
y0, y1 = sg[:, 1].max(), sg[:, 1].min()
x0, y0, x1, y1

## Build the Model

In [ ]:
base_ws = "./temp/mv_voronoi_SI/"
name = "mv"
exe_name = "mf6"
pertim = 365.25 * 30.

In [ ]:
pertim

### flow model

In [ ]:
ws = f"{base_ws}/flow/"
sim = flopy.mf6.MFSimulation(sim_ws=ws, sim_name=name, exe_name=exe_name)
tdis = flopy.mf6.ModflowTdis(sim, time_units="days", perioddata=((pertim, 1, 1.0),))
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="all",
    complexity="simple",
    outer_dvclose=1e-3,
    outer_maximum=200,
    linear_acceleration="bicgstab",
    inner_dvclose=1e-4,
    inner_maximum=200,
    rcloserecord="1e-2 strict",
)
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=name,
    save_flows=True,
    newtonoptions="NEWTON UNDER_RELAXATION",
)
dis = flopy.mf6.ModflowGwfdisv(
    gwf,
    length_units="meters",
    nlay=nlay,
    ncpl=vor.ncpl,
    nvert=vor.nverts,
    top=top_vg,
    botm=botm,
    vertices=vor.get_disv_gridprops()["vertices"],
    cell2d=vor.get_disv_gridprops()["cell2d"],
    idomain=[1, 1, idomain_2, 1, 1, 1],
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=11.0)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    xt3doptions=True,
    save_specific_discharge=True,
    save_saturation=True,
    icelltype=[1, 0, 0, 0, 0, 0],
    k=[kaq_vg, kaq_vg, kclay_vg, kaq_vg, kaq_vg, kaq_vg],
    k33=[0.25 * kaq_vg, 0.25 * kaq_vg, kclay_vg, 0.25 * kaq_vg, 0.25 * kaq_vg, 0.25 * kaq_vg],
)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=rainfall)
evt = flopy.mf6.ModflowGwfevta(
    gwf, surface=top_vg, rate=evaporation, depth=1.0
)
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=welspd, boundnames=True)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    auxiliary=["depth"],
    auxdepthname="depth",
    stress_period_data=drn_spd,
)
sfr = flopy.mf6.ModflowGwfsfr(
    gwf,
    print_stage=True,
    print_flows=True,
    unit_conversion=sfr_unit_conversion,
    stage_filerecord="{}.sfr.stage.bin".format(name),
    budget_filerecord="{}.sfr.cbc".format(name),
    nreaches=len(sfrpak_data),
    packagedata=sfrpak_data,
    connectiondata=sfr_conn,
    perioddata=sfr_spd,
)
lak = flopy.mf6.ModflowGwflak(
    gwf,
    pname="LAK-1",
    time_conversion=lak_time_conversion,
    length_conversion=lak_length_conversion,
    auxiliary=["concentration"],
    print_stage=True,
    print_flows=True,
    stage_filerecord="{}.lak.stage.bin".format(name),
    budget_filerecord="{}.lak.cbc".format(name),
    nlakes=1,
    packagedata=lakpak_data,
    connectiondata=lak_connections,
    perioddata=lake_spd,
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=name + ".hds",
    budget_filerecord=name + ".cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)
# write the model datasets
sim.write_simulation()

# run the model
success = sim.run_simulation()

### Transport model

In [ ]:
pertim, pertim / 360

In [ ]:
ws = f"{base_ws}/transport/"
simt = flopy.mf6.MFSimulation(sim_ws=ws, sim_name=name, exe_name=exe_name)
tdis = flopy.mf6.ModflowTdis(simt, time_units="days", perioddata=((pertim, pertim / (365.25 / 12.), 1.0),))
ims = flopy.mf6.ModflowIms(
    simt,
    print_option="all",
    complexity="simple",
    outer_dvclose=1e-3,
    outer_maximum=200,
    linear_acceleration="bicgstab",
    inner_dvclose=1e-4,
    inner_maximum=200,
    rcloserecord="1e-2 strict",
)
gwt = flopy.mf6.ModflowGwt(
    simt,
    modelname=name,
)
dis = flopy.mf6.ModflowGwtdisv(
    gwt,
    length_units="meters",
    nlay=nlay,
    ncpl=vor.ncpl,
    nvert=vor.nverts,
    top=top_vg,
    botm=botm,
    vertices=vor.get_disv_gridprops()["vertices"],
    cell2d=vor.get_disv_gridprops()["cell2d"],
    idomain = [1, 1, idomain_2, 1, 1, 1],

)
ic = flopy.mf6.ModflowGwtic(gwt, strt=0.0)
adv = flopy.mf6.ModflowGwtadv(
    gwt, scheme="tvd",
)
dsp = flopy.mf6.ModflowGwtdsp(gwt, diffc=0.0e-12, alh=75.0, ath1=7.5)
mst = flopy.mf6.ModflowGwtmst(gwt, porosity=[0.2, 0.2, porosity_2, 0.2, 0.2, 0.2])
pd = [
    ("GWFHEAD", f"../flow/{name}.hds", None),
    ("GWFBUDGET", f"../flow/{name}.cbc", None),
]
fmi = flopy.mf6.ModflowGwtfmi(gwt, packagedata=pd)
sourcerecarray = [
    ("LAK-1", "AUX", "CONCENTRATION"),
]
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray)

oc = flopy.mf6.ModflowGwtoc(
    gwt,
    concentration_filerecord=f"{name}.ucn",
    saverecord=[("CONCENTRATION", "LAST"),],
    printrecord=[("BUDGET", "ALL")],
)
# write the model datasets
simt.write_simulation()

In [ ]:
# run the model
success = simt.run_simulation(silent=True)

In [ ]:
success

### process model results

In [ ]:
lake_stage = float(gwf.lak.output.stage().get_data().squeeze())
lake_stage

In [ ]:
lake_stage_vg = np.full(vor.ncpl, 1e30, dtype=float)
idx = (lake_stage > top_vg) & (lake_cells_vg > 0)
lake_stage_vg[idx] = lake_stage 

In [ ]:
head = gwf.output.head().get_data().squeeze()
head.shape, head.min(), head.max()

In [ ]:
cbc = gwf.output.budget()
cbc.list_unique_records(), cbc.list_unique_packages()

In [ ]:
spdis = cbc.get_data(text="DATA-SPDIS")[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(
    spdis, gwf, head=head
)
qx.shape

In [ ]:
lake_q = cbc.get_data(text="LAK", full3D=True)[0]
lake_q.shape

In [ ]:
lake_q_dir = np.zeros(lake_q.shape, dtype=int)
lake_q_dir[lake_q < 0.0] = -1
lake_q_dir[lake_q > 0.0] = 1

#### plot top of model and results

In [ ]:
top_range = (0, 20)
top_levels = np.arange(0, 25, 5)
head_range = (-1, 5)
head_levels = np.arange(1, head_range[1] + 1, 1)

extent = gwf.modelgrid.extent
xA = well_points[-1][0]
yB = 10500 * ft2m
xsA_pts = ((xA, extent[3]), (xA, extent[2])) 
xsB_pts = ((extent[0], yB), (extent[1], yB))
xsA_line = ((xsA_pts[0][0], xsA_pts[1][0]), (xsA_pts[0][1], xsA_pts[1][1]))
xsB_line = ((xsB_pts[0][0], xsB_pts[1][0]), (xsB_pts[0][1], xsB_pts[1][1]))

In [ ]:
xsA_line, xsA_pts

Plot river mapping

In [ ]:
# dv = 150.0 * ft2m
dv = 100.0  # m
cmap = "viridis"

with styles.USGSMap():
    fig = plt.figure(figsize=two_panel_figsize, constrained_layout=False)
    gs = gridspec.GridSpec(ncols=10, nrows=24, figure=fig)
    ax0 = fig.add_subplot(gs[:, :5])
    ax2 = fig.add_subplot(gs[9:17, 5:])
    ax1 = fig.add_subplot(gs[0:8, 5:], sharex=ax2)
    # ax1.get_shared_x_axes().join(ax1, ax2)
    ax3 = fig.add_subplot(gs[17:, 5:])

    ax = ax0
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=0)
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    mm.plot_array(
        kclay_loc_vg,
        masked_values=[
            0,
        ],
        cmap=clay_cmap,
        alpha=0.5,
    )
    mm.plot_array(
        lake_cells_vg,
        masked_values=[
            0,
        ],
        cmap=lake_cmap,
        alpha=0.5,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    plot_wells(ax, ms=3)
    plot_feature_labels(ax)

    # # cross-section lines
    # dlabel = 400
    # ax.plot(
    #     xsA_line[0],
    #     xsA_line[1],
    #     [voronoi_grid.extent[3], 0],
    #     lw=1,
    #     ls=":",
    #     color="red",
    # )
    # styles.add_annotation(
    #     ax=ax,
    #     text="A",
    #     xy=xsA_pts[0],
    #     xytext=(-15, -15),
    #     textcoords="offset points",
    #     arrowprops=arrowprops,
    #     bold=True,
    #     color="red",
    # )
    # styles.add_annotation(
    #     ax=ax,
    #     text="A'",
    #     xy=xsA_pts[1],
    #     xytext=(-15, 10),
    #     textcoords="offset points",
    #     arrowprops=arrowprops,
    #     bold=True,
    #     color="red",
    # )
    # mm.ax.plot(
    #     xsB_line[0],
    #     xsB_line[1],
    #     lw=1, 
    #     ls=":", 
    #     color="red",
    # )
    # styles.add_annotation(
    #     ax=ax,
    #     text="B",
    #     xy=xsB_pts[0],
    #     xytext=(15, 15),
    #     textcoords="offset points",
    #     arrowprops=arrowprops,
    #     bold=True,
    #     color="red",
    # )
    # styles.add_annotation(
    #     ax=ax,
    #     text="B'",
    #     xy=xsB_pts[1],
    #     xytext=(-15, 10),
    #     textcoords="offset points",
    #     arrowprops=arrowprops,
    #     bold=True,
    #     color="red",
    # )
   
    
    xticks = np.arange(mm.extent[0], mm.extent[1], 1000.0).tolist()
    yticks = np.arange(mm.extent[2], mm.extent[3], 1000.0).tolist()
    set_ticklabels(ax, fmt="{:.0f}", xticks=xticks, yticks=yticks)

    ax = ax1
    ax.set_aspect("equal", "box")
    plt.setp(ax.get_xticklabels(), visible=False)
    styles.heading(ax=ax, idx=1)

    xll, xur = x0 - dv, x1 + dv
    yll, yur = y0 - 2.0 * dv, y0
    xticks = np.arange(xll, xur + 100, 100.0).tolist()
    yticks = np.arange(yll, yur + 100, 100.0).tolist()
    add_subdomain(ax0, xll, xur, yll, yur, text="B")

    mm = flopy.plot.PlotMapView(
        modelgrid=voronoi_grid, ax=ax, extent=(xll, xur, yll, yur)
    )
    mm.plot_array(
        sfr_plt_array,
        lw=0,
        alpha=0.5,
        masked_values=[0],
        cmap=intersection_cmap,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    add_nodes(ax, xll, xur, yll, yur)
    set_ticklabels(ax, yticks=yticks, skip_xlabel=True)

    ax = ax2
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=2)

    xll, xur = x0 - dv, x1 + dv
    yll, yur = y1, y1 + 2.0 * dv
    yticks = np.arange(yll, yur + 100.0, 100.0).tolist()
    add_subdomain(ax0, xll, xur, yll, yur, text="C")

    mm = flopy.plot.PlotMapView(
        modelgrid=voronoi_grid, ax=ax, extent=(xll, xur, yll, yur)
    )
    mm.plot_array(
        sfr_plt_array,
        lw=0,
        alpha=0.5,
        masked_values=[0],
        cmap=intersection_cmap,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    add_nodes(ax, xll, xur, yll, yur)
    set_ticklabels(ax, xticks=xticks, yticks=yticks)

    # legend
    ax = ax3
    xy0 = (-100, -100)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # fake data to set up legend
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="black",
        mec="none",
        mew=0.0,
        label="Cell centroid",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="red",
        mec="none",
        mew=0.0,
        label="Well",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc="cyan",
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="Lake",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc="brown",
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="Confining unit",
    )
    ax.axhline(xy0[0], **river_dict, label="River")
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc=intersection_rgba,
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="River cells",
    )
    # ax.axhline(xy0[0], lw=1, ls=":", color="red", label="Cross-section\nline")
    styles.graph_legend(
        ax,
        ncol=2,
        loc="lower center",
        labelspacing=0.1,
        columnspacing=0.6,
        handletextpad=0.3,
    )

    # save the figure
    fpth = "../doc/figures/mv_voronoi_river_discretization.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
xlims = (voronoi_grid.extent[0], voronoi_grid.extent[1])
xticks = np.arange(
    xlims[0], xlims[1], 1000.0
).tolist()
ylims = (voronoi_grid.extent[2], voronoi_grid.extent[3])
yticks = np.arange(
    ylims[0], ylims[1], 1000.0
).tolist()
xlims, xticks, ylims, yticks

p2_loc = tuple(welspd[-1][0:2])
p2_z = gwf.modelgrid.zcellcenters[p2_loc]

In [ ]:
with styles.USGSMap():
    fig = plt.figure(figsize=two_panel_figsize, constrained_layout=True)
    gs = gridspec.GridSpec(ncols=2, nrows=24, figure=fig)
    ax0 = fig.add_subplot(gs[:22, 0])
    ax1 = fig.add_subplot(gs[:22, 1])
    ax2 = fig.add_subplot(gs[22:, :])

    for ax in (ax0, ax1):
        ax.set_xlim(xlims)
        # ax.set_xticks(xticks)
        ax.set_ylim(ylims)
        # ax.set_yticks(yticks)
        ax.set_aspect("equal", "box")

    # topography
    ax = ax0
    styles.heading(ax=ax, idx=0)
    mm = flopy.plot.PlotMapView(
        model=gwf,
        ax=ax,
        extent=voronoi_grid.extent,
    )
    cb = mm.plot_array(top_vg, vmin=top_range[0], vmax=top_range[1])
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        top_vg,
        **mv_contour_dict,
        levels=top_levels,
    )
    ax.clabel(
        cs,
        **clabel_dict,
    )
    set_ticklabels(ax, fmt="{:.0f}")

    # topography colorbar
    cbar = plt.colorbar(cb, ax=ax, orientation="horizontal", shrink=0.65)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=6,
        pad=2,
    )
    cbar.ax.set_title(
        "Elevation (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # cross-section lines
    dlabel = 400
    ax.plot(
        # [7750.0 * ft2m, 7750.0 * ft2m],
        xsA_line[0],
        xsA_line[1],
        [voronoi_grid.extent[3], 0],
        lw=1,
        ls=":",
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A",
        # xy=(7750 * ft2m, voronoi_grid.extent[3]),
        xy=xsA_pts[0],
        xytext=(-15, -15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A'",
        # xy=(7750 * ft2m, 0),
        xy=xsA_pts[1],
        xytext=(15, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    mm.ax.plot(
        # [0, voronoi_grid.extent[1]], [10500 * ft2m, 10500 * ft2m], 
        xsB_line[0],
        xsB_line[1],
        lw=1, 
        ls=":", 
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B",
        # xy=(0, 10500 * ft2m),
        xy=xsB_pts[0],
        xytext=(15, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        # xy=(voronoi_grid.extent[1], 10500 * ft2m),
        xy=xsB_pts[1],
        xytext=(-15, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # head
    ax = ax1
    styles.heading(ax=ax, idx=1)
    mm = flopy.plot.PlotMapView(
        model=gwf,
        ax=ax,
        extent=voronoi_grid.extent,
    )
    cb = mm.plot_array(head, vmin=head_range[0], vmax=head_range[1])
    # lp = mm.plot_array(lake_q_dir, masked_values=[0], cmap=drain_cmap, edgecolor="none", vmin=0.5, vmax=2.5,)

    mm.plot_grid(**grid_dict)
    q = mm.plot_vector(qx, qy, normalize=False)
    qk = plt.quiverkey(
        q,
        0.97,
        1.02,
        1,
        label="1 m/d",
        labelpos="W",
        labelcolor="black",
        fontproperties={"size": 8},
    )

    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        head,
        **mv_contour_dict,
        levels=head_levels,
    )
    ax.clabel(
        cs,
        **clabel_dict,
    )
    set_ticklabels(ax, fmt="{:.0f}", skip_yticklabels=True, skip_ylabel=True)

    # head colorbar
    cbar = plt.colorbar(cb, ax=ax, orientation="horizontal", shrink=0.65)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=6,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # legend
    ax = ax2
    xy0 = (-100, -100)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # fake data to set up legend
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="red",
        mec="none",
        mew=0.0,
        label="Well",
    )
    ax.axhline(xy0[0], color="cyan", lw=0.5, label="Lake")
    ax.axhline(xy0[0], **river_dict, label="River")
    ax.axhline(xy0[0], lw=1, ls=":", color="red", label="Cross-section line")
    ax.axhline(xy0[0], **contour_label_dict, label="Contour line (m)")
    styles.graph_legend(
        ax,
        ncol=5,
        loc="lower center",
        labelspacing=0.1,
        columnspacing=0.6,
        handletextpad=0.3,
    )

    fpth = "../doc/figures/mv_voronoi_map.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
ylim = (-80, 30)
with styles.USGSMap():
    fig = plt.figure(figsize=one_panel_figsize, constrained_layout=True)
    gs = gridspec.GridSpec(ncols=1, nrows=21, figure=fig)
    ax0 = fig.add_subplot(gs[:11, :])
    ax1 = fig.add_subplot(gs[11:22, :])

    # cross-section A-A'
    ax = ax0
    styles.heading(ax, idx=0)
    fx = flopy.plot.PlotCrossSection(
        model=gwf,
        ax=ax,
        line={
            "line": xsA_pts
        },
    )
    fx.plot_array(head, head=head, vmin=head_range[0], vmax=head_range[1])
    fx.plot_grid(**grid_dict)
    
    fxl = flopy.plot.PlotCrossSection(
        modelgrid=lake_vg_grid,
        ax=ax,
        line={
            "line": xsA_pts
        },
    )
    fxl.plot_array(lake_stage_vg, head=lake_stage_vg, cmap=lake_cmap, masked_values=[1e30], edgecolor="none")
        

    ax.set_ylim(ylim)
    xs_xticks = np.arange(0, ax.get_xlim()[1], 1000)
    ax.set_xticks(xs_xticks, labels=[f"{x/1000:.0f}" for x in xs_xticks])
    ax.set_xlabel("Cross-section distance (km)")
    ax.set_ylabel("Elevation (m)")

    # annotation
    styles.add_annotation(
        ax=ax,
        text="A",
        xy=(0, 6),
        xytext=(8, 4),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A'",
        xy=(ax.get_xlim()[1], 3.),
        xytext=(-15, 12),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text=well_boundnames[-1],
        # xy=(ax.get_xlim()[1] - 2700 * ft2m, -150 * ft2m),
        xy=(ax.get_xlim()[1] - well_points[-1][1], p2_z),
        xytext=(-25, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="Blue Lake",
        # xy=(ax.get_xlim()[1] - 2700 * ft2m, -150 * ft2m),
        xy=(1500, lake_stage),
        xytext=(-25, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="black",
    )

    # cross-section B-B'
    ax = ax1
    styles.heading(ax, idx=1)
    fx = flopy.plot.PlotCrossSection(
        model=gwf,
        ax=ax,
        # line={"line": [(0, 10500 * ft2m), (12500 * ft2m, 10500 * ft2m)]},
        line= {"line": xsB_pts},
    )
    hv = fx.plot_array(head, head=head, vmin=head_range[0], vmax=head_range[1])
    fx.plot_grid(**grid_dict)

    ax.set_ylim(ylim)
    xs_xticks = np.arange(0, ax.get_xlim()[1], 1000)
    ax.set_xticks(xs_xticks, labels=[f"{x/1000:.0f}" for x in xs_xticks])
    ax.set_xlabel("Cross-section distance (km)")
    ax.set_ylabel("Elevation (m)")

    # annotation
    styles.add_annotation(
        ax=ax,
        text="B",
        xy=(0, 43 * ft2m),
        xytext=(12, 8),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        xy=(ax.get_xlim()[1], 12),
        xytext=(-16, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # head colorbar
    cbar = plt.colorbar(hv, ax=ax, orientation="horizontal", shrink=0.65)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=6,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    fig.align_labels()

    fpth = "../doc/figures/mv_voronoi_xsection.png"
    plt.savefig(fpth, dpi=300);

In [ ]:
# fig, axs = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True,
#         figsize=two_panel_figsize,
#         constrained_layout=True,
#         subplot_kw=dict(aspect="equal"),)

with styles.USGSMap():
    fig = plt.figure(figsize=six_panel_figsize, constrained_layout=True)
    gs = gridspec.GridSpec(ncols=3, nrows=24, figure=fig)
    ax0 = fig.add_subplot(gs[:11, 0])
    ax1 = fig.add_subplot(gs[:11, 1])
    ax2 = fig.add_subplot(gs[:11, 2])
    ax3 = fig.add_subplot(gs[11:22, 0])
    ax4 = fig.add_subplot(gs[11:22, 1])
    ax5 = fig.add_subplot(gs[11:22, 2])
    ax6 = fig.add_subplot(gs[22:, :])

    axs = [ax0, ax1, ax2, ax3, ax4, ax5]

    for ax in axs:
        ax.set_xlim(xlims)
        ax.set_xticks(xticks)
        ax.set_ylim(ylims)
        ax.set_yticks(yticks)
        ax.set_aspect("equal", "box")

    skip_x = [True, True, True, False, False, False]
    skip_y = [False, True, True, False, True, True]

    times = gwt.output.concentration().get_times()
    conc = gwt.output.concentration().get_data(totim=times[-1])

    for k in range(6):
        ax = axs[k]
        ax.set_title(f"Layer {k + 1}")
        mm = flopy.plot.PlotMapView(
            model=gwt,
            ax=ax,
            extent=gwt.modelgrid.extent,
            layer=k,
        )
        mm.plot_grid(**grid_dict)

        #plot confining unit
        if k == 2:
            mm.plot_array(
                kclay_loc_vg,
                masked_values=[
                    0,
                ],
                cmap=clay_cmap,
                alpha=0.5,
            )
        plot_wells(ax=ax, ms=3)
        plot_river(ax=ax)
        plot_lake(ax=ax)
        cs = mm.contour_array(conc, 
                              masked_values=[0], 
                              levels=[0.001, 0.01, 0.1, 1], 
                              **mv_contour_dict)

        ax.clabel(
            cs,
            inline=True,
            fmt="%1.3g",
            fontsize=6,
            inline_spacing=0.5,
        )
        set_ticklabels(ax, fmt="{:.0f}", skip_yticklabels=skip_y[k], skip_ylabel=skip_y[k], skip_xticklabels=skip_x[k], skip_xlabel=skip_x[k])
        styles.heading(ax, idx=k)


    # legend
    ax = ax6
    xy0 = (-100, -100)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # fake data to set up legend
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="red",
        mec="none",
        mew=0.0,
        label="Well",
    )
    ax.axhline(xy0[0], color="cyan", lw=0.5, label="Lake")
    ax.axhline(xy0[0], **river_dict, label="River")
    ax.axhline(xy0[0], **contour_label_dict, label="Concentration (mg/L)")
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc="brown",
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="Confining unit",
    )
    styles.graph_legend(
        ax,
        ncol=5,
        loc="lower center",
        labelspacing=0.1,
        columnspacing=0.6,
        handletextpad=0.3,
        frameon=False,
    )

    fpth = "../doc/figures/mv_voronoi_map_concentration.png"
    plt.savefig(fpth, dpi=300);    
    

In [ ]:
gwf.sfr.output.methods()

In [ ]:
gwf.lak.output.methods()

In [ ]:
gwt.output.methods()

In [ ]:
gwf.output.list().get_data(totim=10958, incremental=True)

In [ ]:
gwf.output.budget().list_unique_records()